In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3, ResNet50, DenseNet201
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Input, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [3]:
batchsize = 8
train_datagen = ImageDataGenerator(
    rescale=1./255, rotation_range=0.2, shear_range=0.2, zoom_range=0.2,
    width_shift_range=0.2, height_shift_range=0.2, validation_split=0.2)

train_data = train_datagen.flow_from_directory(
    '/kaggle/input/mydata/train_data/train_data',
    target_size=(80, 80), batch_size=batchsize, class_mode='categorical', subset='training')

validation_data = train_datagen.flow_from_directory(
    '/kaggle/input/mydata/train_data/train_data',
    target_size=(80, 80), batch_size=batchsize, class_mode='categorical', subset='validation')

test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(
    '/kaggle/input/mydata/test_data/test_data',
    target_size=(80, 80), batch_size=batchsize, class_mode='categorical')

Found 54336 images belonging to 2 classes.
Found 13583 images belonging to 2 classes.
Found 16979 images belonging to 2 classes.


In [4]:
def build_model(base_model):
    base_model = base_model(include_top=False, weights='imagenet', input_tensor=Input(shape=(80, 80, 3)))
    x = Flatten()(base_model.output)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(2, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=x)
    for layer in base_model.layers:
        layer.trainable = False
    return model


In [5]:
models = {
    'InceptionV3': build_model(InceptionV3),
    'ResNet50': build_model(ResNet50),
    'DenseNet201': build_model(DenseNet201)
}

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
74836368/74836368 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
best_model_name = None
best_val_acc = 0
best_model = None

for name, model in models.items():
    print(f"Training {name}...")
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
    checkpoint = ModelCheckpoint(f'/kaggle/working/{name}.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True, verbose=3)
    earlystop = EarlyStopping(monitor='val_loss', patience=7, verbose=3, restore_best_weights=True)
    learning_rate = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=3)
    callbacks = [checkpoint, earlystop, learning_rate]
    
    model.fit(train_data, steps_per_epoch=train_data.samples//batchsize, validation_data=validation_data,
              validation_steps=validation_data.samples//batchsize, callbacks=callbacks, epochs=5)
    
    loss_val, acc_val = model.evaluate(validation_data)  

    print(f"Validation Accuracy for {name}: {acc_val}")
    
    if acc_val > best_val_acc:
        best_val_acc = acc_val
        best_model_name = name
        best_model = model

print(f"Best model: {best_model_name} with accuracy: {best_val_acc}")

Training InceptionV3...
Epoch 1/5
6791/6792 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9338 - loss: 0.1732
Epoch 1: val_loss improved from inf to 0.23777, saving model to /kaggle/working/InceptionV3.weights.h5
6792/6792 ━━━━━━━━━━━━━━━━━━━━ 283s 40ms/step - accuracy: 0.9338 - loss: 0.1732 - val_accuracy: 0.9083 - val_loss: 0.2378 - learning_rate: 0.0010
Epoch 2/5

Epoch 2: val_loss improved from 0.23777 to 0.03762, saving model to /kaggle/working/InceptionV3.weights.h5
6792/6792 ━━━━━━━━━━━━━━━━━━━━ 3s 423us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0376 - learning_rate: 0.0010
Epoch 3/5
6789/6792 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.9340 - loss: 0.1672
Epoch 3: val_loss did not improve from 0.03762
6792/6792 ━━━━━━━━━━━━━━━━━━━━ 210s 31ms/step - accuracy: 0.9340 - loss: 0.1672 - val_accuracy: 0.8906 - val_loss: 0.2609 - learning_rate: 0.0010
Epoch 4/5

Epoch 4: val_loss did not improve from 0.03762
6792/6792 ━━━━━━━━━━━━━━━━━━━

In [10]:
# Save best model
best_model.save('/kaggle/working/best_model.h5')
print("Best model saved successfully")

# Verify file
if os.path.exists('/kaggle/working/best_model.h5'):
    print("File exists at:", os.getcwd() + '/best_model.h5')

!zip -r zip_folder.zip /kaggle/working/best_model.h5
from IPython.display import FileLink
FileLink('zip_folder.zip')

Best model saved successfully
File exists at: /kaggle/working/best_model.h5
  adding: kaggle/working/best_model.h5 (deflated 16%)


/kaggle/working/zip_folder.zip